# MATH3976 Project Draft - Simple Circuit Simulator

David Rapisarda

SID: 440429320

### What's currently been completed

Most of the plumbing required for connecting and manipulating circuits is completed, as well as a rough but working implementation of linear DC operating point analysis (linearly controlled voltage and current sources aren't implemented yet, and there are still some unhandled cases where an invalid circuit can break the solver). 

An example of a simple circuit that currently does not work is in "operating point test 2" in tests/op.jl.

### What will be implemented in the final submission

 - More robust DC operating point analysis: either checking for invalid circuit configurations (hard) or replacing source components with a 'non-ideal' equivalent (resistor in series for voltage source, resistor in parallel for current source) so that entering an invalid circuit (at least an invalid linear circuit) is impossible
 - Non-linear DC operating point analysis with diodes, bi-polar junction transistors (BJTs), and metal oxide field effect transistors (MOSEFTs) using the Newton-Raphson method or a similar iterative process - aim to be consistent with the 'hand analysis' models taught in ELEC courses, rather than the experimental data based approach taken by other simulators
 - Transient analysis for linear circuits with just resistors, capacitors and inductors, and for non-linear circuits with combinations of diodes, MOSFETs and BJTs - most likely implement by casting the system of differential equations into a form usable by the the various `ode` solver functions
 
With the above features implemented, various circuits will be simulated, and the results compared with both hand analysis and SPICE simulations.

In [7]:
include("SimpleCircuits.jl")
using SimpleCircuits

## DC operating point: example

Consider the following circuit:

<img src="img/circuit1.png" />

Solving the circuit here is equivalent to solving a system with two unknowns, $v_1$ and $v_2$. The linear system can be obtained using Kirchhoff's Current Law at node 2 and the definition of a voltage source at node 1. Using $V = IR$ and summing the currents at node 2:

$$ \frac{v_1 - v_2} {5000} + \frac {0 - v_2} {10000} = 0 \implies \frac{1}{5000} v_1 - (\frac{1}{5000} + \frac{1}{10000}) v_2 = 0$$

And at node 1:

$$ v_1 = v_{GND} + 5 = 5 $$

This process for constructing the linear system of equations can be generalized for any linear circuit. Analysis with the current implementation (see `op.jl` for details):

In [2]:
# testing linear DC operating point
circ = Circuit()

# create the components
r1 = Resistor(5e+3)
r2 = Resistor(10e+3)
v_DC = DCVoltageSource(5.)

# connect the components, and specify ground (reference voltage)
connect!(circ, v_DC.pHigh, r1.p1, "Node 1")
connect!(circ, r1.p2, r2.p1, "Node 2")
connect!(circ, r2.p2, v_DC.pLow)
connect!(circ, circ.gnd, v_DC.pLow)

# compute the DC operating point
node_voltages = op(circ)

Dict{SimpleCircuits.Node,Float64} with 3 entries:
  Node: GND…               => 0.0
  Node: Node 2…            => 3.333333333333333
  Node: Node 1…            => 5.0